---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [1]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import re

In [3]:
lines = [line.rstrip('\n') for line in open('university_towns.txt')]

rows = []
for line in lines:
    state_groups = re.split(r'\[', line)
    region_groups = re.split(r' \(', line)
    if 'edit]' in state_groups :
        state = state_groups[0]
    else:
        rows.append([state, region_groups[0]])

pd.DataFrame(rows, columns=['State', 'RegionName']).head()


,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo


In [7]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''

    lines = [line.rstrip('\n') for line in open('university_towns.txt')]

    rows = []
    for line in lines:
        state_groups = re.split(r'\[', line)
        region_groups = re.split(r' \(', line)
        if 'edit]' in state_groups :
            state = state_groups[0]
        else:
            rows.append([state, region_groups[0]])

    return pd.DataFrame(rows, columns=['State', 'RegionName'])
    
# get_list_of_university_towns()

In [71]:
def get_recssion():
#     gdp_annual = (pd.read_excel('gdplev.xls',  skiprows=5, usecols='A:C', skip_footer=191)[2:]
#                  .rename(columns={'Unnamed: 0':'Year'}))
#     gdp_annual

    gdp_quarterly = (pd.read_excel('gdplev.xls',  skiprows=5, usecols='E:G')[2:]
                 .rename(columns={'Unnamed: 0':'Quarter', 'GDP in billions of chained 2009 dollars' : 'GDP'}))[['Quarter', 'GDP']]

    # set DateTimeIndex to Quarter
    gdp_quarterly = gdp_quarterly.set_index(pd.DatetimeIndex(gdp_quarterly['Quarter']))

    # Consider only values after 2000 Q1
    gdp_quarterly = gdp_quarterly[gdp_quarterly.index >= '2000-01-01']

    # add quarterly difference of GDP
    newcol = [i for i in np.diff(gdp_quarterly['GDP']) ]
    newcol.insert(0, 0)

    gdp_quarterly = gdp_quarterly.assign(Diff = newcol)


#     rec_index = 0
    for i in range(4,gdp_quarterly.shape[0]):
        # continuous quarters values
        q1 = gdp_quarterly.iloc[i-4]['Diff']
        q2 = gdp_quarterly.iloc[i-3]['Diff']
        q3 = gdp_quarterly.iloc[i-2]['Diff']
        q4 = gdp_quarterly.iloc[i-1]['Diff']

        # compare if there is decline and growth
        if((q1 < 0) & (q2 < 0) & (q3 >0) & (q4 > 0) ):
#             print(str(q1) + ', ' +  str(q2) + ', ' + str(q3) + ', ' + str(q4))
#             print(gdp_quarterly.iloc[i-4]['Quarter'] + ', ' + gdp_quarterly.iloc[i-3]['Quarter'] + ', ' +
#                  gdp_quarterly.iloc[i-2]['Quarter']+ ', ' + gdp_quarterly.iloc[i-1]['Quarter'])
            # get the row index and break
            recession_index = i
            break

    recession = gdp_quarterly.iloc[recession_index-4 : recession_index]
  
    return recession

get_recssion()

,Quarter,GDP,Diff
Quarter,,,
2009-01-01,2009q1,14375.0,-202.0
2009-04-01,2009q2,14355.6,-19.4
2009-07-01,2009q3,14402.5,46.9
2009-10-01,2009q4,14541.9,139.4


In [80]:
rec = get_recssion()
# rec.iloc[-1].Quarter
rec[rec['GDP'] == np.min(rec['GDP'])].Quarter.iloc[0]

'2009q2'

In [76]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    rec = get_recssion()

    return rec.iloc[0].Quarter

# get_recession_start()

'2009q1'

In [77]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    rec = get_recssion()   
    return rec.iloc[-1].Quarter

# get_recession_end()

'2009q4'

In [81]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    rec = get_recssion()
    return rec[rec['GDP'] == np.min(rec['GDP'])].Quarter.iloc[0]

# get_recession_bottom()

'2009q2'

In [83]:
def get_gdp_quarterly():
    gdp_quarterly = (pd.read_excel('gdplev.xls',  skiprows=5, usecols='E:G')[2:]
                 .rename(columns={'Unnamed: 0':'Quarter', 'GDP in billions of chained 2009 dollars' : 'GDP'}))[['Quarter', 'GDP']]

    # set DateTimeIndex to Quarter
    gdp_quarterly = gdp_quarterly.set_index(pd.DatetimeIndex(gdp_quarterly['Quarter']))

    # Consider only values after 2000 Q1
    gdp_quarterly = gdp_quarterly[gdp_quarterly.index >= '2000-01-01']

    # add quarterly difference of GDP
    newcol = [i for i in np.diff(gdp_quarterly['GDP']) ]
    newcol.insert(0, 0)

    gdp_quarterly = gdp_quarterly.assign(Diff = newcol)
    return gdp_quarterly

In [98]:
univ_towns = get_list_of_university_towns()
gdp = get_gdp_quarterly()
# gdp
univ_towns
# gdp[gdp.index > '2007']
# univ_towns

,Quarter,GDP,Diff
Quarter,,,
2007-04-01,2007q2,14838.7,112.7
2007-07-01,2007q3,14938.5,99.8
2007-10-01,2007q4,14991.8,53.3
2008-01-01,2008q1,14889.5,-102.3
2008-04-01,2008q2,14963.4,73.9
2008-07-01,2008q3,14891.6,-71.8
2008-10-01,2008q4,14577.0,-314.6
2009-01-01,2009q1,14375.0,-202.0
2009-04-01,2009q2,14355.6,-19.4


In [ ]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    return "ANSWER"

In [166]:
df = pd.read_csv('City_Zhvi_AllHomes.csv')
# select columns
cols = df.columns[1:3].append(df.columns[51:])
# project columns
df1 = df[cols]
# set multi-index
df1 = df1.set_index(['State', 'RegionName'])

# Transpose to convert to DateTimeIndex
df2 = df1.T
df3 = df2.set_index(pd.DatetimeIndex(df2.index))

# df4 = df2.set_index(df3.index.to_period(freq='Q'))
#df3

# sum months to Quarterly GDP values
df3 = df3.resample('Q', how='sum')

# set Quarters as Index
df4 = df3.set_index(df3.index.to_period(freq='Q'))

# transpose DF to make quarters as columns
df5 = df4.T

# rename columns to string 2000q1 like values
df5.columns = df5.columns.strftime('%Fq%q')
df5



c:\softwares\python\python36-32\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).sum()


,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
NY,New York,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1546400.0,1568400.0,1584200.0,1596800.0,1622400.0,1671600.0,1718500.0,1748600.0,1774900.0,1174400.0
CA,Los Angeles,621200.0,643400.0,662900.0,678500.0,699000.0,717300.0,735200.0,759100.0,785900.0,818100.0,...,1494100.0,1527200.0,1556600.0,1586400.0,1614500.0,1641800.0,1673200.0,1698100.0,1732400.0,1168100.0
IL,Chicago,415200.0,430900.0,443600.0,456400.0,470800.0,485400.0,499200.0,511300.0,526500.0,532700.0,...,577900.0,587300.0,603800.0,603200.0,618100.0,624900.0,623700.0,618200.0,624600.0,424000.0
PA,Philadelphia,159000.0,160900.0,162400.0,164100.0,166000.0,166600.0,168800.0,172600.0,177400.0,182200.0,...,341200.0,345900.0,347000.0,348600.0,353900.0,363700.0,366600.0,370300.0,380800.0,257400.0
AZ,Phoenix,335500.0,343100.0,348000.0,352200.0,358800.0,364700.0,368100.0,372900.0,379600.0,385100.0,...,492800.0,496100.0,505500.0,514600.0,522500.0,537200.0,551500.0,563700.0,574300.0,390400.0
NV,Las Vegas,397800.0,403100.0,406200.0,411000.0,418600.0,425200.0,430100.0,438400.0,448000.0,452800.0,...,510200.0,520200.0,526400.0,532500.0,544800.0,560300.0,571900.0,583800.0,591600.0,399900.0
CA,San Diego,668700.0,703100.0,736300.0,768100.0,801600.0,828800.0,853500.0,875800.0,903700.0,938600.0,...,1440600.0,1467100.0,1489300.0,1510100.0,1536200.0,1559300.0,1576400.0,1588000.0,1608700.0,1079500.0
TX,Dallas,253400.0,251600.0,254600.0,263500.0,269200.0,267900.0,267200.0,272700.0,277700.0,281400.0,...,319900.0,326700.0,334600.0,341100.0,363400.0,385700.0,403800.0,421500.0,433800.0,298600.0
CA,San Jose,1122800.0,1219700.0,1295600.0,1366500.0,1412000.0,1410600.0,1370400.0,1336700.0,1324300.0,1373300.0,...,2038200.0,2091100.0,2144800.0,2194300.0,2270200.0,2329200.0,2367400.0,2410800.0,2456800.0,1644400.0


In [ ]:

def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"